In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2

In [24]:
from make_password import password  # Import the password function fromm a seperte file
pw = password # insert whatever your database password is here
db = "Statcast"  # Database name
host = "localhost"  # Database host
user = "postgres"  # Database user
port = "5432"  # Database port

def connecting():
    #Establish a connection to the PostgreSQL database.

    connection = None  # Initialize connection variable
    connection = psycopg2.connect(
        database=db,
        host=host,
        user=user,
        password=pw,
        port=port
    )  # Establish connection to the database
    return connection  # Return the connection object

def execute_query(connection, query):
 
    # Execute a SQL query on the database.
    try:
        cursor = connection.cursor()  # Create a cursor object
        cursor.execute(query)  # Execute the SQL query
        connection.commit()  # Commit the transaction
    except psycopg2.Error as e:
        print(f"Error executing query: {e}")  # Print error message
        connection.rollback()  # Rollback the transaction in case of error
    finally:
        cursor.close()  
def read_query(connection, query):
    """
    Execute a SQL query and fetch all results. 
    Returns:
        result (list): List of tuples containing the query results.
    """
    cursor = connection.cursor()  # Create a cursor object
    result = None  # Initialize result variable
    cursor.execute(query)  # Execute the SQL query
    result = cursor.fetchall()  # Fetch all results
    return result  # Return the query results

What is a Whiff?

A whiff in baseball is a very straight forward concept, it's when a batter swings at a pitch and misses. While simple, it is also one of the most important stats in modern baseball. Understanding how and how often pitchers get whiffs is crucial to effectivley evalulate pitchers. A pitcher who is unable to miss bats rarley gets very far in the league. A quick look at the leaderboard for whiffs in 2024 is quite revealing.

In [27]:
# The following code as well as many others in this notebook reference a statcast_2024 database. This was obtained using the pybaseball api and was imported into a postgres database directly in pgadmin4 as the code to do so is quite long and tedious. 
# All CSVS can be found under the CSV folder on github.

get_whiffs = '''SELECT player_name, COUNT(*) as whiff_count
FROM statcast_2024 
WHERE description = 'swinging_strike'
GROUP BY player_name
ORDER BY COUNT(*) DESC;'''

connection = connecting() 
execute_query(connection, get_whiffs) 
whiff_leadder_board = read_query(connection, get_whiffs)
whiff_leadder_board = pd.DataFrame(whiff_leadder_board, columns=['player_name', 'whiff_count'])
print(whiff_leadder_board.head(10))  # Display the top 10 players with the most whiffs



        player_name  whiff_count
0      Cease, Dylan          432
1      Ragans, Cole          424
2     Skubal, Tarik          399
3     Rodón, Carlos          395
4   Peralta, Freddy          389
5    Gilbert, Logan          386
6  Crochet, Garrett          373
7     Wheeler, Zack          371
8      Ober, Bailey          367
9       Sale, Chris          367


Those are some pretty good names topping the leaderboard, including both 2024 AL and NL CY Young winners, as well as a host of elite pitchers. It's pretty obvious a high amount of whiffs is genreally the sign of an elite pitcher. But how well do they correlate with a pitchers run prevention ability, which is the baseline job of a pitcher. For this we will look at whiff rate (simply # of whiffs / # of pitchers) for all qualified  starting pitchers from 2024 and compare to their ERA+ (in order to take park factors into account).
